In [1]:
class eigen:
    
    def __init__(self):  
        
        #variables
        sig=0.001# fermi coeficiente
        eh=0.381 #EnergyofH
        self.a0=100  #bohr Radius Angstrons
        self.L=200/self.a0
        self.Ry= eh##/m
        n=3000
        
        self.n = n
        self.x = self.xgrade()
        self.sig = sig
    
    
    def xgrade(self):
        return np.cos(np.pi*np.arange(self.n+1)/self.n) 
    
    def fpot(self,xc):
        return np.array(1/(np.exp((self.x-xc)/self.sig**2)+1))
    
    def pot(self,Eqw,lqw):
        lqw=(lqw/2)
        fl=self.fpot(-lqw)
        fr=self.fpot(lqw)
        return (1+fl-fr)*Eqw
    
    def mass(self,masswell,massbar,masswidth):
        masswidth=masswidth/self.L/self.a0
        delta_mass=massbar-masswell
        return self.pot(delta_mass,masswidth)+masswell
         

    
    def massae(self,val,mw1,EQW, Eg):
        Ec=1700 #meV 
        Ev=5500 #meV
        #mb1=(Eb/Ew)*mw1 # condição usado por Nelson 
        
        mb1=mw1/0.4
        
        mw= mw1*(1+val/Eg)
        
        #mb=mb1*(1-(EQW-val)/(Eg+Ec+Ev))
        mb=mw/0.4
        return mw,mb
      
    def chebb(self,m,Eqw ,Lqw): #########################################
        
        lqw= Lqw/self.L/self.a0
        Eqw = Eqw/self.Ry; #[Eqw] = meV
        
        
        M=1/m    
                
        xrep=np.matlib.repmat(self.x,self.n+1,1).T
        c=np.matrix(np.insert((-1)**np.arange(1,self.n),(0,self.n-1),[2,2*(-1)**self.n]))
        dx= xrep-xrep.T
        D=(np.dot(c.T,1/c))/(dx+(np.eye(self.n+1)))
        D= D-np.diagflat(np.sum(D,1))
        

        #mD^2
     
        
        DD=np.dot(D,D)#3º1/m*(D*D)
        mDD=np.dot(np.diagflat(M),DD)
        
        #DmD
        
        h=(-(self.x[2:]-self.x[0:-2]))**2
        diag1= np.append(0,((1/h[:])*(M[2:]-M[0:-2])))
        diag2= np.append((-(1/h[:])*(M[2:]-M[0:-2])),0)
        DmD=np.diagflat([diag1], 1)+np.diagflat([diag2], -1)
        
        D_total=(mDD+DmD)[1:self.n,1:self.n]
        
        Vc=self.pot(Eqw,lqw)[1:self.n]*self.Ry
        H=-self.Ry*D_total/self.L/self.L+np.diagflat(Vc) # diagflat(Vc) - produz uma matriz diagonal com vetor Vc
        
        #H=-self.Ry*mDD/self.L/self.L+np.diagflat(Vc)
        
        #pegando os eigenvalues e eigenfunctions do Hamiltoniano com o Ry=cte
        val,vet=la.eig(H)
        p=np.argsort(val) # ordenando os eigenvalues 
        valp=val[p] 
        vetp=vet[:,p]

        
        f_wave=10000*np.power(vetp,2)+np.matlib.repmat(valp,self.n-1,1)
        
        return valp,vetp,f_wave,Vc
    
    def get_mE(self,m,mw1, Eqw ,lqw,Eg):
       
        Eold=300
        e=[]
        diff=[]
        dif=2
        count=0
        for i in range(1):
            while dif>=.2:
                val,vet,wave,Vc= self.chebb(m,Eqw,lqw)
                E=val[0]
                dif=np.abs(Eold - E)/Eold
                Eold=E
                e.append(E)
                diff.append(dif)
                mw,mb=self.massae(E,mw1,Eqw,Eg)
                m=self.mass(mw,mb,lqw)
                count+=1
                if count==15:
                    print(E,mw)
        return E,e,diff,m,mw,mb
        

In [2]:
import numpy as np
import numpy.matlib
from numpy import linalg as la
import matplotlib.pyplot as plt

# Apenas chebb (0.105,0.22)#


In [3]:
tst=eigen()
lqw=[6.40, 12.73, 18.95, 25.22]
Eg=[]
for i in range(len(lqw)):
    me=tst.mass(0.105,0.262,lqw[i])
    mh=tst.mass(0.22,0.55,lqw[i])
    vb,vetv,wv,potv = tst.chebb(me,Eqw=900,Lqw=lqw[i])
    vbc,vetc,wvc,potvc = tst.chebb(mh,Eqw=2900,Lqw=lqw[i])
    Eg.append((vb[0]+vbc[0])/1000+1.7)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


In [4]:
Eg

[3.156291790450032, 2.433487044980285, 2.1637348923838338, 2.017191428762696]

# Apenas chebb BAMA traore#

In [ ]:
tst=eigen()
lqw=[6.40, 12.73, 18.95, 25.22]
Eg=1700
Egap=[]
Vwm=[]
Cwm=[]

for i in range(len(lqw)):
    me=tst.mass(0.35,0.87,lqw[i])
    mh=tst.mass(0.47,1.175,lqw[i])
    E,e,dif ,m,mw,mb=tst.get_mE(m=me,mw1=0.35, Eqw=2700 ,lqw=lqw[i],Eg=Eg)
    Eh,eh,diffh,mh,mwh,mbh=tst.get_mE(m=mh,mw1=0.47, Eqw=2800 ,lqw=lqw[i], Eg=Eg)    
    Egap.append((E+Eh+Eg)/1000)#+1.7)
    Cwm.append([mw,mb])
    Vwm.append([mwh,mbh])

In [ ]:
Egap

# Apenas chebb (0.35,0.47)#


In [5]:
tst=eigen()
lqw=[6.40, 12.55, 18.95, 25.22]
Eg2=[]
for i in range(len(lqw)):
    me=tst.mass(0.35,0.87,lqw[i])
    mh=tst.mass(0.47,1.175,lqw[i])
    vb,vetv,wv,potv = tst.chebb(me,Eqw=900,Lqw=lqw[i])
    vbc,vetc,wvc,potvc = tst.chebb(mh,Eqw=2900,Lqw=lqw[i])
    Eg2.append((vb[0]+vbc[0])/1000+1.7)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


In [6]:
Eg2

[2.681806251523486,
 2.14502439812433,
 1.9509982373800958,
 1.8622265172800614,
 1.8159060797718838]

# Apenas chebb poço Infinto

In [ ]:
tst=eigen()
lqw=[5, 10 ,15 ,20]
EgQW=[]
for i in range(len(lqw)):
    me=tst.mass(0.4,0.4,lqw[i])
    vb,vetv,wv,potv = tst.chebb(me,Eqw=9000000000,Lqw=lqw[i])
    EgQW.append(vb[0])

In [ ]:
EgQW

In [ ]:
a0=100
eh=0.381#meV
m=0.4
LQW=[5, 10 ,15 ,20]
Ry=eh/m
n=1#np.arange(1,7,1)
eg=[]
for i in range(len(LQW)):
    exata=((n**2*np.pi**2)/((LQW[i]/a0)**2))*Ry
    eg.append(exata)

In [ ]:
eg

# Com não-parabolicidade \ Eg constante


In [ ]:
tst=eigen()
lqw=[6.40, 12.73, 18.95, 25.22]
Eg=1700
Egap=[]
Vwm=[]
Cwm=[]

for i in range(len(lqw)):
    me=tst.mass(0.35,0.87,lqw[i])
    mh=tst.mass(0.47,1.175,lqw[i])
    E,e,dif ,m,mw,mb=tst.get_mE(m=me,mw1=0.35, Eqw=900 ,lqw=lqw[i],Eg=Eg)
    Eh,eh,diffh,mh,mwh,mbh=tst.get_mE(m=mh,mw1=0.47, Eqw=2900 ,lqw=lqw[i], Eg=Eg)    
    Egap.append((E+Eh+Eg)/1000)#+1.7)
    Cwm.append([mw,mb])
    Vwm.append([mwh,mbh])

In [ ]:
Egap

# Com não-parabolicidade \ Eg variável

In [17]:
tst=eigen()
lqw=[6.4,12.73,18.95,25.35,31.59,37.99,50.79,63.59,76.39,89.19,95.59]
Eg=[110,280,220,160,107.8,85.1,56.9,40.1,28.8, 20.5,17.1]
Egap=[]
Vwm=[]
Cwm=[]

for i in range(len(lqw)):
    me=tst.mass(0.35,0.87,lqw[i])
    mh=tst.mass(0.47,1.175,lqw[i])
    E,e,dif ,m,mw,mb=tst.get_mE(m=me,mw1=0.35, Eqw=900 ,lqw=lqw[i],Eg=Eg[i]+1700)
    Eh,eh,diffh,mh,mwh,mbh=tst.get_mE(m=mh,mw1=0.47, Eqw=2900 ,lqw=lqw[i], Eg=Eg[i]+1700)    
    Egap.append((E+Eh+Eg[i]+1700)/1000)#+1.7)
    Cwm.append([mw,mb])
    Vwm.append([mwh,mbh])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


In [42]:
E

9.196032746126075

In [40]:
np.savetxt('e.out', T, delimiter=';',fmt='%1.3f') 

In [39]:
T

array([[0.601, 1.502],
       [0.528, 1.32 ],
       [0.501, 1.253],
       [0.491, 1.227],
       [0.485, 1.212],
       [0.481, 1.202],
       [0.477, 1.192],
       [0.474, 1.186],
       [0.473, 1.183],
       [0.472, 1.181],
       [0.472, 1.18 ]])